# Basic XRayLabTool Examples

This notebook demonstrates basic usage of XRayLabTool for X-ray optical property calculations.

## Setup and Installation

First, ensure XRayLabTool is installed:

In [ ]:
# Install XRayLabTool if not already installed
# !pip install xraylabtool

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import xraylabtool as xrt

print(f"XRayLabTool version: {xrt.__version__}")

## Example 1: Single Material Calculation

Calculate X-ray properties for silicon at 8 keV:

In [ ]:
# Calculate properties for silicon
result = xrt.calculate_single_material_properties(
    formula="Si",
    density=2.33,  # g/cm³
    energy=8000,  # eV
)

print(f"Material: {result.formula}")
print(f"Density: {result.density_g_cm3} g/cm³")
print(f"Energy: {result.energy_ev} eV")
print(f"Wavelength: {result.wavelength_angstrom:.3f} Å")
print("")
print("Refractive index decrement:")
print(f"  δ (delta): {result.delta:.2e}")
print(f"  β (beta):  {result.beta:.2e}")
print("")
print(f"Critical angle: {result.critical_angle_degrees:.3f}°")
print(f"Critical angle: {result.critical_angle_mrad:.2f} mrad")
print(f"Attenuation length: {result.attenuation_length_cm:.2f} cm")
print(f"Linear absorption coefficient: {result.linear_absorption_coefficient:.3f} cm⁻¹")

## Example 2: Compound Materials

Calculate properties for silicon dioxide (quartz):

In [ ]:
# Silicon dioxide (quartz)
sio2_result = xrt.calculate_single_material_properties(
    formula="SiO2",
    density=2.20,  # g/cm³ for quartz
    energy=8000,
)

print(f"Material: {sio2_result.formula}")
print(f"Critical angle: {sio2_result.critical_angle_degrees:.3f}°")
print(f"Attenuation length: {sio2_result.attenuation_length_cm:.2f} cm")

# Compare with pure silicon
print("\nComparison with pure Si:")
print(f"Si critical angle:   {result.critical_angle_degrees:.3f}°")
print(f"SiO2 critical angle: {sio2_result.critical_angle_degrees:.3f}°")
print(
    f"Difference: {result.critical_angle_degrees - sio2_result.critical_angle_degrees:.3f}°"
)

## Example 3: Energy Range Analysis

Study how properties vary with X-ray energy:

In [ ]:
# Define energy range
energies = np.logspace(3, 4.5, 50)  # 1 keV to ~32 keV

# Calculate properties for each energy
results_si = []
for energy in energies:
    result = xrt.calculate_single_material_properties("Si", 2.33, energy)
    results_si.append(result)

# Extract arrays for plotting
critical_angles = [r.critical_angle_mrad for r in results_si]
attenuation_lengths = [r.attenuation_length_cm for r in results_si]
delta_values = [r.delta for r in results_si]
beta_values = [r.beta for r in results_si]

print(f"Energy range: {energies[0]:.0f} - {energies[-1]:.0f} eV")
print(
    f"Critical angle range: {min(critical_angles):.2f} - {max(critical_angles):.2f} mrad"
)
print(
    f"Attenuation length range: {min(attenuation_lengths):.1f} - {max(attenuation_lengths):.1f} cm"
)

### Visualization of Energy Dependence

In [ ]:
# Create subplots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 10))

# Critical angle vs energy
ax1.loglog(energies, critical_angles, "b-", linewidth=2)
ax1.set_xlabel("Energy (eV)")
ax1.set_ylabel("Critical Angle (mrad)")
ax1.set_title("Critical Angle vs Energy")
ax1.grid(True, alpha=0.3)

# Attenuation length vs energy
ax2.loglog(energies, attenuation_lengths, "r-", linewidth=2)
ax2.set_xlabel("Energy (eV)")
ax2.set_ylabel("Attenuation Length (cm)")
ax2.set_title("Attenuation Length vs Energy")
ax2.grid(True, alpha=0.3)

# Delta vs energy
ax3.loglog(energies, delta_values, "g-", linewidth=2)
ax3.set_xlabel("Energy (eV)")
ax3.set_ylabel("δ (delta)")
ax3.set_title("Refractive Index Decrement (δ)")
ax3.grid(True, alpha=0.3)

# Beta vs energy
ax4.loglog(energies, beta_values, "m-", linewidth=2)
ax4.set_xlabel("Energy (eV)")
ax4.set_ylabel("β (beta)")
ax4.set_title("Absorption Parameter (β)")
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Example 4: Material Comparison

Compare different materials for X-ray optics applications:

In [ ]:
# Define materials for comparison
materials = [
    {"name": "Silicon", "formula": "Si", "density": 2.33},
    {"name": "Silicon Dioxide", "formula": "SiO2", "density": 2.20},
    {"name": "Aluminum", "formula": "Al", "density": 2.70},
    {"name": "Copper", "formula": "Cu", "density": 8.96},
    {"name": "Gold", "formula": "Au", "density": 19.32},
]

energy = 8000  # eV
comparison_results = []

for material in materials:
    result = xrt.calculate_single_material_properties(
        material["formula"], material["density"], energy
    )
    comparison_results.append(
        {
            "Material": material["name"],
            "Formula": material["formula"],
            "Density (g/cm³)": material["density"],
            "Critical Angle (°)": result.critical_angle_degrees,
            "Critical Angle (mrad)": result.critical_angle_mrad,
            "Attenuation Length (cm)": result.attenuation_length_cm,
            "δ (×10⁶)": result.delta * 1e6,
            "β (×10⁶)": result.beta * 1e6,
        }
    )

# Create DataFrame for nice display
df = pd.DataFrame(comparison_results)
print(f"Material comparison at {energy} eV:")
print(df.round(3).to_string(index=False))

### Visualization of Material Comparison

In [ ]:
# Bar plot comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

materials_names = [m["name"] for m in materials]
critical_angles_deg = df["Critical Angle (°)"].values
attenuation_lengths = df["Attenuation Length (cm)"].values

# Critical angles
bars1 = ax1.bar(
    materials_names,
    critical_angles_deg,
    color=["blue", "lightblue", "green", "orange", "gold"],
)
ax1.set_ylabel("Critical Angle (degrees)")
ax1.set_title(f"Critical Angles at {energy} eV")
ax1.tick_params(axis="x", rotation=45)
ax1.grid(True, alpha=0.3)

# Add value labels on bars
for bar, value in zip(bars1, critical_angles_deg, strict=False):
    ax1.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.001,
        f"{value:.3f}°",
        ha="center",
        va="bottom",
        fontsize=9,
    )

# Attenuation lengths (log scale)
bars2 = ax2.bar(
    materials_names,
    attenuation_lengths,
    color=["blue", "lightblue", "green", "orange", "gold"],
)
ax2.set_ylabel("Attenuation Length (cm)")
ax2.set_title(f"Attenuation Lengths at {energy} eV")
ax2.set_yscale("log")
ax2.tick_params(axis="x", rotation=45)
ax2.grid(True, alpha=0.3)

# Add value labels on bars
for bar, value in zip(bars2, attenuation_lengths, strict=False):
    ax2.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() * 1.2,
        f"{value:.1f} cm",
        ha="center",
        va="bottom",
        fontsize=9,
    )

plt.tight_layout()
plt.show()

## Example 5: Complex Formulas

Demonstrate parsing of complex chemical formulas:

In [ ]:
# Complex formulas
complex_materials = [
    {"formula": "Ca5(PO4)3F", "density": 3.18, "name": "Fluorapatite"},
    {"formula": "CuSO4·5H2O", "density": 2.29, "name": "Copper sulfate pentahydrate"},
    {"formula": "Al2(SO4)3·18H2O", "density": 1.69, "name": "Aluminum sulfate hydrate"},
]

print("Complex formula parsing and calculation:")
print("=" * 60)

for material in complex_materials:
    # First, let's parse the formula to see the composition
    from xraylabtool.utils import parse_formula

    composition = parse_formula(material["formula"])
    print(f"\n{material['name']}: {material['formula']}")
    print(f"Elemental composition: {composition}")

    # Calculate properties
    result = xrt.calculate_single_material_properties(
        material["formula"], material["density"], 8000
    )

    print(f"Density: {material['density']} g/cm³")
    print(f"Critical angle: {result.critical_angle_degrees:.3f}°")
    print(f"Attenuation length: {result.attenuation_length_cm:.2f} cm")
    print("-" * 40)

## Example 6: Batch Processing

Process multiple materials efficiently:

In [ ]:
# Create a list of materials for batch processing
batch_materials = [
    {"formula": "Si", "density": 2.33},
    {"formula": "SiO2", "density": 2.20},
    {"formula": "Al", "density": 2.70},
    {"formula": "Al2O3", "density": 3.95},
    {"formula": "Cu", "density": 8.96},
    {"formula": "Au", "density": 19.32},
    {"formula": "C", "density": 2.26},  # Graphite
    {"formula": "Fe", "density": 7.87},
]

# Define energy points
energy_points = [5000, 8000, 10000, 15000]  # eV

# Batch calculation
batch_results = xrt.calculate_xray_properties(batch_materials, energy_points)

print("Batch processing completed:")
print(f"Materials processed: {len(batch_materials)}")
print(f"Energy points: {len(energy_points)}")
print(f"Total calculations: {len(batch_results)}")

# Display sample results
print("\nSample results (first 10):")
for i, result in enumerate(batch_results[:10]):
    print(
        f"{i+1:2d}. {result.formula:6s} at {result.energy_ev:5.0f} eV: "
        f"θc = {result.critical_angle_degrees:.3f}°, "
        f"att_len = {result.attenuation_length_cm:6.2f} cm"
    )

## Example 7: Unit Conversions

Demonstrate energy-wavelength conversions:

In [ ]:
from xraylabtool.utils import energy_to_wavelength, wavelength_to_energy

# Energy to wavelength conversions
energies_kev = [1, 5, 8, 10, 15, 20, 30]
energies_ev = [e * 1000 for e in energies_kev]

print("Energy to Wavelength Conversions:")
print("Energy (keV)  Energy (eV)  Wavelength (Å)")
print("-" * 40)

for kev, ev in zip(energies_kev, energies_ev, strict=False):
    wavelength = energy_to_wavelength(ev)
    print(f"{kev:8.0f}      {ev:8.0f}      {wavelength:10.4f}")

# Wavelength to energy conversions
print("\nCommon X-ray lines:")
xray_lines = [
    ("Cu Kα1", 1.5406),
    ("Mo Kα1", 0.7093),
    ("Ag Kα1", 0.5594),
    ("Cr Kα1", 2.2909),
]

print("Line      Wavelength (Å)  Energy (eV)  Energy (keV)")
print("-" * 50)

for line_name, wavelength in xray_lines:
    energy_ev = wavelength_to_energy(wavelength)
    energy_kev = energy_ev / 1000
    print(f"{line_name:8s}  {wavelength:12.4f}  {energy_ev:9.1f}  {energy_kev:9.3f}")

## Summary

This notebook demonstrated:

1. **Single material calculations** - Basic usage for simple materials
2. **Compound materials** - Parsing complex chemical formulas
3. **Energy dependence** - How properties vary with X-ray energy
4. **Material comparison** - Side-by-side analysis of different materials
5. **Complex formulas** - Handling hydrated compounds and complex stoichiometry
6. **Batch processing** - Efficient calculation for multiple materials
7. **Unit conversions** - Energy-wavelength relationships

### Key Insights:

- **Critical angles** are typically small (< 1°) for hard X-rays
- **Heavy elements** (Au, Cu) have larger critical angles than light elements (Si, C)
- **Attenuation lengths** vary dramatically between materials and energies
- **Energy scaling**: Critical angle ∝ E⁻¹, Attenuation length ∝ E³
- **Compounds** have properties intermediate between their constituent elements

### Next Steps:

- Explore the CLI interface with `xraylabtool --help`
- Try the synchrotron applications examples
- Investigate material optimization for specific applications
- Learn about the underlying X-ray physics in the documentation